### Import libraries and data

In [44]:
import sympy as sp
import numpy as np
import itertools as it
import json
from scipy.linalg import lu, lu_factor, lu_solve 


candidates = json.load(open("data1.json", "r"))
lookup = [[1,-1], [1,1], [1,0], [0,1], [0,0]]
solns = [
    [1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1],
   #['u1s1', 'u1s2', 'u1t1', 'u1t2', 'u2s1', 'u2s2', 'u2t1', 'u2t2', 'v1s1', 'v1s2', 'v1t1', 'v1t2', 'v2s1', 'v2s2', 'v2t1', 'v2t2']
]
npSols = np.matrix(solns)
singleSols = np.array(list(it.product([-1,0,1], repeat = 7)))
print(npSols.shape)

(4, 16)


### Generate parities

In [7]:
singleParities = tuple(it.product([-1,1], repeat=7))
pars = tuple(it.product(singleParities, repeat=2))

def lup(x):
    return list(lookup[i].copy() for i in x)
def appParity(x,y):
    return list(
        [i[0]*j, i[1]*j] for (i,j) in zip(x,y)
        )

def appParPair(uvst, parPair):
    return {'u' : lup(uvst['u']), 's' : lup(uvst['s']),
            'v' : appParity(lup(uvst['v']), parPair[0]),
            't' : appParity(lup(uvst['t']), parPair[1]),
            'par' : parPair, 'orig':uvst}

### Generate Matrices

In [8]:
def tensor(x,y):
    return list(
    i*j for (i,j) in it.product(x,y)
    )

def matGen(uvst):
    left = tuple(i + j for (i,j) in zip(uvst['u'], uvst['v']))
    right = tuple(i + j for (i,j) in zip(uvst['s'], uvst['t'])) 
    return ( 
        uvst['par'], 
        sp.Matrix(
            list(tensor(i,j) for (i,j) in zip(left, right)) 
            + solns.copy()
        ).transpose()
      )
def genListEntries(uvst):
    for i in pars: 
        yield matGen(appParPair(uvst,i))

def genList(uvst):
    return list(genListEntries(uvst))
    

In [9]:
%timeit genList(candidates[0])

1 loops, best of 3: 9.9 s per loop


###Check functions

In [10]:
def checkSoln(mat):
    return (len(mat[1]) == 7) and (max(mat[1]) < 7)

def getValid(cands):
    return [i for (i,j) in cands if checkSoln(j.rref())]

In [22]:
a = genList(candidates[0])
a[0]

(((-1, -1, -1, -1, -1, -1, -1), (-1, -1, -1, -1, -1, -1, -1)), Matrix([
 [ 1,  1,  0,  0,  0,  0,  0, 1, 0, 0, 0],
 [ 1,  0,  1,  0,  0,  0,  0, 0, 1, 0, 0],
 [ 0, -1, -1,  0,  0, -1,  0, 0, 0, 0, 0],
 [ 0,  0,  0,  0,  0,  0,  0, 0, 0, 0, 0],
 [-1,  0,  0,  0,  0,  0,  0, 0, 0, 1, 0],
 [-1,  0,  0,  1,  0,  0,  0, 0, 0, 0, 1],
 [ 0,  0,  0,  0,  0,  0,  0, 0, 0, 0, 0],
 [ 0,  0,  0,  0,  0,  0,  0, 0, 0, 0, 0],
 [ 0,  0,  0,  0,  0,  0,  0, 0, 0, 0, 0],
 [ 0,  0,  0,  0,  0,  0,  0, 0, 0, 0, 0],
 [ 0,  0,  0,  0,  0,  1,  1, 1, 0, 0, 0],
 [ 0,  0,  0,  0,  0,  0, -1, 0, 1, 0, 0],
 [ 0,  0,  0,  0,  0,  0,  0, 0, 0, 0, 0],
 [ 0,  0, -1, -1, -1,  0,  0, 0, 0, 0, 0],
 [ 0,  0,  1,  0,  0,  0,  1, 0, 0, 1, 0],
 [ 0,  0,  0,  0,  1,  0, -1, 0, 0, 0, 1]]))

In [13]:
results = [(i,getValid(genList(i))) for i in candidates]

In [14]:
a = [len(j) for (i,j) in results]
len([i for i in a if i])

40

# Final Results

In [25]:
results

[({'s': [1, 2, 3, 3, 3, 4, 4],
   't': [4, 2, 2, 4, 3, 2, 0],
   'u': [0, 2, 2, 3, 4, 2, 4],
   'v': [4, 4, 3, 3, 3, 2, 1]},
  []),
 ({'s': [1, 2, 3, 3, 3, 4, 4],
   't': [4, 2, 2, 4, 3, 2, 1],
   'u': [0, 2, 2, 3, 4, 2, 4],
   'v': [4, 4, 3, 3, 3, 2, 0]},
  [((-1, -1, -1, -1, -1, -1, -1), (-1, 1, -1, -1, 1, -1, -1)),
   ((-1, -1, -1, -1, -1, -1, -1), (-1, 1, -1, -1, 1, -1, 1)),
   ((-1, -1, -1, -1, -1, -1, -1), (-1, 1, -1, -1, 1, 1, -1)),
   ((-1, -1, -1, -1, -1, -1, -1), (-1, 1, -1, -1, 1, 1, 1)),
   ((-1, -1, -1, -1, -1, -1, -1), (-1, 1, -1, 1, 1, -1, -1)),
   ((-1, -1, -1, -1, -1, -1, -1), (-1, 1, -1, 1, 1, -1, 1)),
   ((-1, -1, -1, -1, -1, -1, -1), (-1, 1, -1, 1, 1, 1, -1)),
   ((-1, -1, -1, -1, -1, -1, -1), (-1, 1, -1, 1, 1, 1, 1)),
   ((-1, -1, -1, -1, -1, -1, -1), (1, 1, -1, -1, 1, -1, -1)),
   ((-1, -1, -1, -1, -1, -1, -1), (1, 1, -1, -1, 1, -1, 1)),
   ((-1, -1, -1, -1, -1, -1, -1), (1, 1, -1, -1, 1, 1, -1)),
   ((-1, -1, -1, -1, -1, -1, -1), (1, 1, -1, -1, 1, 1, 1)),
   ((-1

I'm pretty sure these all have some sort of latt

#Restrict to [-1,0,1]

This doesn't actually get rid of any results, luckily.
You can just ignore them.

In [94]:
def appParPair2(uvst, parPair):
    return {'u' : lup(uvst['u']), 's' : lup(uvst['s']),
            'v' : appParity(lup(uvst['v']), parPair[0]),
            't' : appParity(lup(uvst['t']), parPair[1]),
            'par' : parPair}

def appPars2(uvst):
    for i in uvst[1]:
        yield appParPair2(uvst[0], i)

def getPars2(uvst):
    return list(appPars2(uvst))

def matGen2(uvst):
    left = tuple(i + j for (i,j) in zip(uvst['u'], uvst['v']))
    right = tuple(i + j for (i,j) in zip(uvst['s'], uvst['t'])) 
    return ( 
        uvst['par'], 
        singleSols * np.matrix(
            list(tensor(i,j) for (i,j) in zip(left, right)) 
        )
      )

def checkRow(r):
    return np.array([np.array_equal(r,i) for i in npSols])

def matCheck2(matr):
    a1 = np.array(list(checkRow(r) for r in matr)).transpose()
    a2 = np.array([i.any() for i in a1])
    return a2.all()

def matsToCheck(uvst):
    return [matGen2(i) for i in appPars2(uvst)]


def goodPars(uvst):
    return [i for (i,j) in matsToCheck(uvst) if matCheck2(j)]


at = list(matGen2(i) for i in appPars2(results[1]))
a2 = matsToCheck(results[1])

In [95]:
%timeit goodPars(results[1])

1 loops, best of 3: 2min 17s per loop


In [96]:
finRes = [(i, goodPars((i,j))) for (i,j) in results]

In [98]:
[len(j) for (i,j) in finRes]

[0,
 512,
 512,
 512,
 512,
 0,
 512,
 512,
 0,
 512,
 0,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 0,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 0,
 512,
 512,
 512,
 512,
 0,
 512,
 512,
 512,
 0,
 0,
 512,
 512,
 0,
 512,
 512,
 512,
 0,
 512,
 0,
 512,
 512]